In [4]:
from scipy.stats.mstats import trimmed_mean
import numpy as np
import tensorflow as tf

In [25]:
a = np.array([0.11, 0.55, 1, .17, 0.22, 1, .1, 1])

b = np.round(a, 1)

print(a, b, sep='\n')

[0.11 0.55 1.   0.17 0.22 1.   0.1  1.  ]
[0.1 0.6 1.  0.2 0.2 1.  0.1 1. ]
